## Group Project: EPA FE Analysis and Prediction

### Context: 


### Objective : 

* To identify the different factors that affect fuel economy in surveyed vehicles
* To make a model to predict if an employee will attrite or not


### Dataset :
The data contains (replace - demographic details), (replace - work-related metrics) and (replace - attrition flag).

* **Metric** - Definition


### Import the necessary libraries

In [21]:
# Note: go through later and removed unused dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

#algorithms to use
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

#for tuning the model
from sklearn.model_selection import GridSearchCV

#to ignore warnings
import warnings
warnings.filterwarnings("ignore")

### Read the dataset
Additionally, perform some cleaning and some column combination/drops as agreed upon during prior inspection of the data pool.

In [85]:
#reading the dataset

# use the commented out df definition to import only the columns specified in the above list
df = pd.read_csv('table_export_unclean.csv')

# drop columns decided to be unwanted for our analysis
drops = ['Real-World MPG_City','Real-World MPG_Hwy','Real-World CO2 (g/mi)','Real-World CO2_City (g/mi)',
        'Real-World CO2_Hwy (g/mi)','HP/Weight (lbs)','Ton-MPG (Real-World)','Transmission - Other','Fuel Delivery - Carbureted',
        'Fuel Delivery - Throttle Body Injection','Powertrain - Fuel Cell Vehicle (FCV)','Powertrain - Other (incl. CNG)','4 or Fewer Gears',
        '5 Gears','6 Gears','7 Gears','8 Gears','9 or More Gears']


df.drop(columns = drops)



# erroneous entries requiring replacement
df = df.replace('-', np.NaN)
df = df.replace('Prelim. 2021', '2021')

# attempt to convert columns from object to numeric type wherever possible - df.infer_objects() wasn't cutting it
cols = df.columns
for col in cols:
    try:
        df[col] = pd.to_numeric(df[col])
    except:
        pass

# combine two transmission columns and two powertrain columns, dropping the originals
df['Transmission - Automatic + Lockup'] = df['Transmission - Automatic'] + df['Transmission - Lockup']
df['Powertrain - PHEV and Gas Hybrid'] = df['Powertrain - Plug-in Hybrid Electric Vehicle (PHEV)'] + df['Powertrain - Gasoline Hybrid']
df.drop(columns = ['Transmission - Automatic','Transmission - Lockup','Powertrain - Plug-in Hybrid Electric Vehicle (PHEV)','Powertrain - Gasoline Hybrid'])

df.head()

,Manufacturer,Model Year,Regulatory Class,Vehicle Type,Production (000),Production Share,2-Cycle MPG,Real-World MPG,Real-World MPG_City,Real-World MPG_Hwy,...,Average Number of Gears,Transmission - CVT (Non-Hybrid),4 or Fewer Gears,5 Gears,6 Gears,7 Gears,8 Gears,9 or More Gears,Transmission - Automatic + Lockup,Powertrain - PHEV and Gas Hybrid
0,All,1975,All,All,10224.0,1.0,15.33300,13.05970,12.01552,14.61167,...,NaN,NaN,0.129,0.010,NaN,NaN,NaN,NaN,0.770,NaN
1,All,1976,All,All,12334.0,1.0,16.70719,14.22136,13.18117,15.73946,...,NaN,NaN,0.788,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All,1977,All,All,14123.0,1.0,17.70553,15.06743,14.00580,16.60587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,All,1978,All,All,14448.0,1.0,18.60678,15.83777,14.68193,17.52390,...,NaN,NaN,0.702,0.073,NaN,NaN,NaN,NaN,0.774,NaN
4,All,1979,All,All,13882.0,1.0,18.69594,15.91271,14.87711,17.39245,...,3.3,NaN,0.938,0.062,NaN,NaN,NaN,NaN,0.754,NaN


### Dataset Info

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170 entries, 0 to 5169
Data columns (total 54 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Manufacturer                                         5170 non-null   object 
 1   Model Year                                           5170 non-null   int64  
 2   Regulatory Class                                     5170 non-null   object 
 3   Vehicle Type                                         5170 non-null   object 
 4   Production (000)                                     3870 non-null   float64
 5   Production Share                                     3870 non-null   float64
 6   2-Cycle MPG                                          3972 non-null   float64
 7   Real-World MPG                                       3972 non-null   float64
 8   Real-World MPG_City                                  3972 non-null  

## Exploratory Data Analysis and Data Preprocessing

In [24]:
df.Manufacturer.unique()

array(['BMW', 'Ford', 'GM', 'All', 'Honda', 'Hyundai', 'Toyota', 'VW',
       'Mazda', 'Stellantis', 'Subaru', 'Tesla', 'Mercedes', 'Nissan',
       'Kia'], dtype=object)